## Prototype Pyresample AreaDefinition to_cf()

This notebook is a prototype for a routine that is the counterpart of [load_cf_area()](https://pyresample.readthedocs.io/en/latest/howtos/geometry_utils.html#loading-from-netcdf-cf). It takes a pyresample AreaDefinition object as input and returns an xarray Dataset that holds the basic information needed to prepare netCDF/CF files for this particular grid. It returns a template that can be further worked upon (add relevant variables, more attributes, more dimensions, etc...).

Status:
* Sept 7th 2023 : Prototype ready for inspection. It does not support all possible CF projections. It also does not support user configuration (e.g. via a cf_info).

In [1]:
import numpy as np
import xarray as xr
import pyresample as pr

In [2]:
# load an area_def from a netCDF/CF file (could be any other solution to obtain an AreaDefinition)
ds = xr.open_dataset('https://thredds.met.no/thredds/dodsC/osisaf/met.no/reprocessed/ice/conc_450a_files/monthly/2020/ice_conc_nh_ease2-250_cdr-v3p0_202011.nc')
area_def, cf_info = pr.utils.load_cf_area(ds)

/home/thomasl/mambaforge/envs/py310/lib/python3.10/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
/home/thomasl/mambaforge/envs/py310/lib/python3.10/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)


In [3]:
def to_cf_template(area_def, skip_lonlat=True):
    """Return a template xarray Dataset holding the structure of a netCDF/CF file for this grid."""
    # prepare the crs object
    crs_cf = area_def.crs.to_cf()
    type_of_grid_mapping = crs_cf['grid_mapping_name']
    
    # prepare the x and y axis (1D)
    xy = dict()
    xy_dims = ('x', 'y')
    for axis in xy_dims:
        
        # access the valid standard_names (also handle the 'default')
        try:
            valid_coord_standard_names = pr.utils.cf._valid_cf_coordinate_standardnames[type_of_grid_mapping][axis]
        except KeyError:
            valid_coord_standard_names = pr.utils.cf._valid_cf_coordinate_standardnames['default'][axis]
        
        xy[axis] = dict()
        # CF wants the center of the grid cells
        if axis == 'x':
            xy[axis]['_coords'] = area_def.projection_x_coords
        else:
            xy[axis]['_coords'] = area_def.projection_y_coords     
        # each axis requires a valid name, which depends on the type of projection
        xy[axis]['standard_name'] = valid_coord_standard_names[0]
        # CF recommendation to have axis= attribute
        xy[axis]['axis'] = axis.upper()
        # units
        xy[axis]['units'] = 'm'
    
    # latitude and longitude (2D)
    lons, lats = area_def.get_lonlats()
    
    # define a Dataset as a template.
    #   we cannot define a Dataset without a data variable. The strategy is to 
    #   create the Dataset with an (empty) variable, and the user can later 
    #   add his own variables, and finally drop our 'template' variable before
    #   writing to file.
    varn = 'template'
    shape = lons.shape
    da_empty_data = np.ones_like(lons) * np.nan
    da_dims = list(xy_dims)
    da_coords = {'x':('x',xy['x']['_coords']), 'y':('y',xy['y']['_coords']),}
    if not skip_lonlat:
        da_coords['lon']=(('x','y'),lons)
        da_coords['lat']=(('x','y'),lats)
    ds = xr.Dataset(data_vars={varn:(da_dims,da_empty_data),
                               'crs':([], 0)},
                    coords=da_coords,)
    
    # add CF attributes to the xarray template
    
    # x and y dims
    for axis in xy_dims:
        for attr in xy[axis].keys():
            if attr.startswith('_'): continue
            ds[axis].attrs[attr] = xy[axis][attr]
    
    # crs object
    ds['crs'].attrs = crs_cf
    
    # latitude and longitude
    if not skip_lonlat:
        ds['lon'].attrs={'long_name': 'longitude coordinate', 'standard_name': 'longitude', 'units': 'degrees_east'}
        ds['lat'].attrs={'long_name': 'latitude coordinate', 'standard_name': 'latitude', 'units': 'degrees_north'}
    
    # the empty variable itself
    ds[varn].attrs['grid_mapping'] = 'crs'
    
    # provide an encoder for .to_netcdf()
    nc_encoder = {'crs':{'dtype':'int32'}, 'x':{'_FillValue':None}, 'y':{'_FillValue':None},}
    if not skip_lonlat:
        nc_encoder['lat']={'_FillValue':None}
        nc_encoder['lon']={'_FillValue':None}
    
    return ds, nc_encoder
    

In [4]:
# Demonstrate the routine.

# Use to_cf_template() to obtain a template for the netCDF/CF file
ds, encoder = to_cf_template(area_def, skip_lonlat=False)

# Create a DataArray holding user data that we want to add to the file. 
myData = np.arange(0,ds['lon'].size).reshape(ds['lon'].shape).astype('float')
da_var2 = xr.DataArray(myData, coords=ds['template'].coords, dims=ds['template'].dims,
                       attrs=ds['template'].attrs, name='my_var')
# Add some attributes
da_var2.attrs['long_name'] = 'my variable defined on that particular grid'
da_var2.attrs['units'] = 'K'

# Add the variable to the template file
ds = ds.merge(da_var2)

# We don't need the template variable anymore, drop it.
ds = ds.drop('template')

# Write to file.
o_nc = 'cf_template.nc'
ds.to_netcdf('cf_template.nc', encoding=encoder)

In [5]:
# Simple test: can we load the AreaDefinition from the file?
new_area_def, new_cf_info = pr.utils.load_cf_area(o_nc)
# If yes, is it the same AreaDefinition as we wrote in?
if (area_def != new_area_def):
    print("EQUALITY AREA_EXTENT: ", np.allclose(area_def.area_extent, new_area_def.area_extent))
    print("EQUALITY SHAPE: ", area_def.shape == new_area_def.shape)
    print("EQUALITY CRS: ", area_def.crs == new_area_def.crs)
    lons, lats = area_def.get_lonlats()
    nlons, nlats = new_area_def.get_lonlats()
    print("EQUALITY LATs: ", abs(lats-nlats).max() < 1.e-4)
    print("EQUALITY LONs: ", abs(lons-nlons).max() < 1.e-4)
else:
    print("EQUALITY")

EQUALITY


/home/thomasl/mambaforge/envs/py310/lib/python3.10/site-packages/pyproj/crs/crs.py:1296: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)
